In [1]:
# pip install langchain langchain-community
# pip install langchain-openai tiktoken
# pip install neo4j

#### Neo4jVector

**create vector from langchain documents**

In [3]:
from langchain.docstore.document import Document

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

from langchain_community.vectorstores import Neo4jVector

from langchain_openai import OpenAIEmbeddings

In [5]:
loader = TextLoader(file_path="./data/state_of_the_union.txt", encoding="utf-8")

In [6]:
documents = loader.load()

In [7]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

In [8]:
docs = text_splitter.split_documents(documents)

In [9]:
len(docs)

42

In [10]:
docs[0]

Document(metadata={'source': './data/state_of_the_union.txt'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their de

In [11]:
embeddings = OpenAIEmbeddings()

In [12]:
NEO4J_URI     = 'neo4j+s://264d8780.databases.neo4j.io'
NEO4J_USERNAME= 'neo4j'
NEO4J_PASSWORD= '5l2648jhBn6kzOFi_XcK_yzYCVFzZIpoOPW7xp7M_Ss'
AURA_INSTANCEID= '264d8780'
AURA_INSTANCENAME = 'Instance01'

In [13]:
from neo4j import GraphDatabase

In [14]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [15]:
def delete_all_nodes_and_relationships():
    query = """
    MATCH (n)
    DETACH DELETE n
    """
    with driver.session() as session:
        session.run(query)
    print("All nodes and relationships deleted.")

# Call the function
delete_all_nodes_and_relationships()

All nodes and relationships deleted.


In [17]:
with driver.session() as session:
    
    result = session.run("SHOW INDEXES;")
    
    for record in result:
        print(record)

<Record id=3 name='constraint_1dc138a' state='ONLINE' populationPercent=100.0 type='RANGE' entityType='NODE' labelsOrTypes=['Chunk'] properties=['id'] indexProvider='range-1.0' owningConstraint='constraint_1dc138a' lastRead=neo4j.time.DateTime(2024, 12, 10, 3, 8, 46, 915000000, tzinfo=<UTC>) readCount=770>
<Record id=2 name='index-state-union' state='ONLINE' populationPercent=100.0 type='VECTOR' entityType='NODE' labelsOrTypes=['Chunk'] properties=['embedding'] indexProvider='vector-2.0' owningConstraint=None lastRead=neo4j.time.DateTime(2024, 12, 10, 3, 7, 4, 760000000, tzinfo=<UTC>) readCount=9>
<Record id=0 name='index_343aff4e' state='ONLINE' populationPercent=100.0 type='LOOKUP' entityType='NODE' labelsOrTypes=None properties=None indexProvider='token-lookup-1.0' owningConstraint=None lastRead=neo4j.time.DateTime(2024, 12, 10, 3, 10, 20, 90000000, tzinfo=<UTC>) readCount=4>
<Record id=1 name='index_f7700477' state='ONLINE' populationPercent=100.0 type='LOOKUP' entityType='RELATION

In [18]:
# The Neo4jVector Module will connect to Neo4j and create a vector index if needed.
# 1. Connect to Neo4j
# 2. Document Embedding
# 3. Stores the embedded vectors as properties of nodes in the database, along with any associated metadata
# 2. Create Vector Index

db = Neo4jVector.from_documents(
    documents  = docs, 
    embedding  = OpenAIEmbeddings(model="text-embedding-3-small"), 
    url        = NEO4J_URI, 
    username   = NEO4J_USERNAME, 
    password   = NEO4J_PASSWORD,
    index_name = "index_state_union"
)

In [19]:
query = "What did the president say about Ketanji Brown Jackson"
docs_with_score = db.similarity_search_with_score(query, k=2)

In [20]:
docs_with_score

[(Document(metadata={'source': './data/state_of_the_union.txt'}, page_content='And for our LGBTQ+ Americans, let’s finally get the bipartisan Equality Act to my desk. The onslaught of state laws targeting transgender Americans and their families is wrong. \n\nAs I said last year, especially to our younger transgender Americans, I will always have your back as your President, so you can be yourself and reach your God-given potential. \n\nWhile it often appears that we never agree, that isn’t true. I signed 80 bipartisan bills into law last year. From preventing government shutdowns to protecting Asian-Americans from still-too-common hate crimes to reforming military justice. \n\nAnd soon, we’ll strengthen the Violence Against Women Act that I first wrote three decades ago. It is important for us to show the nation that we can come together and do big things. \n\nSo tonight I’m offering a Unity Agenda for the Nation. Four big things we can do together.  \n\nFirst, beat the opioid epidemi

#### Hybrid Search

Hybrid search combines `vector search` with `fulltext search` with re-ranking and de-duplication of the results.

In [21]:
db = Neo4jVector.from_documents(
    documents = docs, 
    embedding = OpenAIEmbeddings(model="text-embedding-3-small"), 
    url      = NEO4J_URI, 
    username = NEO4J_USERNAME, 
    password = NEO4J_PASSWORD,
    search_type = 'hybrid',
    index_name = "index_state_union"
)

In [22]:
query = "What did the president say about Ketanji Brown Jackson"
docs_with_score = db.similarity_search_with_score(query, k=2)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: 'CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN node.`text` AS text, score, node {.*, `text`: Null, `embedding`: Null, id: Null } AS metadata'


In [23]:
docs_with_score

[(Document(metadata={'source': './data/state_of_the_union.txt'}, page_content='And for our LGBTQ+ Americans, let’s finally get the bipartisan Equality Act to my desk. The onslaught of state laws targeting transgender Americans and their families is wrong. \n\nAs I said last year, especially to our younger transgender Americans, I will always have your back as your President, so you can be yourself and reach your God-given potential. \n\nWhile it often appears that we never agree, that isn’t true. I signed 80 bipartisan bills into law last year. From preventing government shutdowns to protecting Asian-Americans from still-too-common hate crimes to reforming military justice. \n\nAnd soon, we’ll strengthen the Violence Against Women Act that I first wrote three decades ago. It is important for us to show the nation that we can come together and do big things. \n\nSo tonight I’m offering a Unity Agenda for the Nation. Four big things we can do together.  \n\nFirst, beat the opioid epidemi

#### Metadata filtering
Metadata filtering enhances vector search by allowing searches to be refined based on specific node properties. This integrated approach ensures more precise and relevant search results by leveraging both the vector similarities and the contextual attributes of the nodes.

In [24]:
db = Neo4jVector.from_documents(
    documents = docs, 
    embedding = OpenAIEmbeddings(model="text-embedding-3-small"), 
    url      = NEO4J_URI, 
    username = NEO4J_USERNAME, 
    password = NEO4J_PASSWORD,
    #search_type = 'hybrid',
    index_name = "index_state_union"
)

In [25]:
docs[0]

Document(metadata={'source': './data/state_of_the_union.txt'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their de

In [26]:
query  = "What did the president say about Ketanji Brown Jackson"
filter = {"name": {"$eq": "adam"}}

docs = db.similarity_search(query, filter=filter)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: name)} {position: line: 1, column: 152, offset: 151} for query: 'CYPHER runtime = parallel parallelRuntimeSupport=all MATCH (n:`Chunk`) WHERE n.`embedding` IS NOT NULL AND size(n.`embedding`) = toInteger(1536) AND n.`name` = $param_1 WITH n as node, vector.similarity.cosine(n.`embedding`, $embedding) AS score ORDER BY score DESC LIMIT toInteger($k) RETURN node.`text` AS text, score, node {.*, `text`: Null, `embedding`: Null, id: Null } AS metadata'


In [27]:
docs

[]

#### Neo4j Graph
The Neo4j Graph integration is a wrapper for the Neo4j Python driver. It allows querying and updating the Neo4j database in a simplified manner from LangChain. Many integrations allow you to use the Neo4j Graph as a source of data for LangChain.

In [28]:
from langchain_neo4j import Neo4jGraph

In [31]:
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

In [32]:
QUERY = """
"MATCH (m:Movie)-[:IN_GENRE]->(:Genre {name:$genre})
RETURN m.title, m.plot
ORDER BY m.imdbRating DESC LIMIT 5"
"""

In [33]:
graph.query(QUERY, genre="action")

TypeError: Neo4jGraph.query() got an unexpected keyword argument 'genre'